In [3]:
import pandas as pd
import numpy as np 
from tqdm import tqdm

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [4]:
num_games = 4
season_1819 = pd.read_csv(f'NewData/AVG(18-19)[{num_games}].csv').dropna().drop('Unnamed: 0', axis = 1).reset_index(drop= True)
season_1920 = pd.read_csv(f'NewData/AVG(19-20)[{num_games}].csv').dropna().drop('Unnamed: 0', axis = 1).reset_index(drop = True)




In [5]:
def time_series_single(df, season):
    unique_players = df.Name.value_counts().index
    col =  ['MP', 'FG', 'FGA', '3P', '3PA',
            'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK',
            'PTS', 'FDP']
    t_col = ['FDP'] 
    #updater_column = ['Name', 'Date', 'FDP'] + avg_col
    timesteps = 10
    final_df = pd.DataFrame(columns = ['Name', 'Date', 'FDP'] + [str(i) for i in list(range(1,timesteps+1))[::-1]])
    pbar = tqdm(unique_players, desc = f'Single Feature: Converting DF to timeseries ({season})')
    for player in pbar: 
        new_df = df[df.Name == player].sort_values(by = ['Date']).reset_index(drop = True)
        for idx in range(len(new_df)):

            row = new_df.iloc[idx]

            if idx <= timesteps-1: 
                continue 
            last_games = new_df[idx-timesteps: idx]
            l_games = [] 
            for i in range(len(last_games)): 
                l_row = last_games.iloc[i]
                fdp = l_row[t_col].values[0]
                
                l_games.append(fdp)
            
            l_games = l_games[::-1]
            row_dict = {'Name': player, 'Date': row['Date'], 'FDP': row['FDP']}
            for i in range(timesteps): 
                t_dict = {str(i+1): l_games[i]}
                row_dict.update(t_dict)
            final_df = final_df.append(row_dict, ignore_index = True)
            
        
        
    final_df.to_csv(f'NewData/TimeSeriesSingleFeatures({season})[{num_games}].csv', index = False)
    return final_df
             
        
time_series_single_1819 = time_series_single(season_1819, '18-19')   
time_series_single_1920 = time_series_single(season_1920, '19-20')    


time_series_single_1920.head()

Single Feature: Converting DF to timeseries (18-19): 100%|██████████| 455/455 [05:27<00:00,  1.39it/s]
Single Feature: Converting DF to timeseries (19-20): 100%|██████████| 444/444 [03:30<00:00,  2.11it/s]


,Name,Date,FDP,10,9,8,7,6,5,4,3,2,1
0,Kent Bazemore,20191119.0,17.6,19.0,25.7,16.7,16.9,7.6,32.6,25.5,18.3,9.4,29.3
1,Kent Bazemore,20191121.0,20.0,25.7,16.7,16.9,7.6,32.6,25.5,18.3,9.4,29.3,17.6
2,Kent Bazemore,20191123.0,2.0,16.7,16.9,7.6,32.6,25.5,18.3,9.4,29.3,17.6,20.0
3,Kent Bazemore,20191125.0,6.6,16.9,7.6,32.6,25.5,18.3,9.4,29.3,17.6,20.0,2.0
4,Kent Bazemore,20191127.0,30.0,7.6,32.6,25.5,18.3,9.4,29.3,17.6,20.0,2.0,6.6


In [6]:
def time_series_multiple(df, season):
    unique_players = df.Name.value_counts().index
    col =  ['MP', 'FG', 'FGA', '3P', '3PA',
            'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK',
            'PTS', 'FDP']
    avg_col = ['FDP'] + [f'{num_games}_{i}' for i in col]
    updater_column = ['Name', 'Date', 'FDP'] + avg_col
    timesteps = 10
    final_df = pd.DataFrame(columns = ['Name', 'Date', 'FDP'] + [str(i) for i in list(range(1,timesteps+1))[::-1]])
    pbar = tqdm(unique_players, desc = f'Multiple: Converting DF to timeseries ({season})')
    for player in pbar:
        new_df = df[df.Name == player].sort_values(by = ['Date']).reset_index(drop = True)
        #print(new_df)
        for idx in range(len(new_df)):

            row = new_df.iloc[idx]

            if idx <= timesteps-1: 
                continue 
            last_games = new_df[idx-timesteps: idx]
            l_games = [] 
            for i in range(len(last_games)): 
                l_row = last_games.iloc[i]
                avg_array = np.array(l_row[avg_col])
                
                l_games.append(avg_array)
            
            l_games = l_games[::-1]
            row_dict = {'Name': player, 'Date': row['Date'], 'FDP': row['FDP']}#.update({'hello': 0})
            for i in range(timesteps): 
                t_dict = {str(i+1): l_games[i]}
                row_dict.update(t_dict)
            final_df = final_df.append(row_dict, ignore_index = True)
            
        
    final_df.to_csv(f'NewData/TimeSeriesMultFeatures({season})[{num_games}].csv', index = False)
    return final_df
             
        
time_series_multiple_1819 = time_series_multiple(season_1819, '18-19')   
time_series_multiple_1920 = time_series_multiple(season_1920, '19-20')    

time_series_multiple_1920.head()

Multiple: Converting DF to timeseries (19-20): 100%|██████████| 444/444 [05:26<00:00,  1.36it/s]


,Name,Date,FDP,10,9,8,7,6,5,4,3,2,1
0,Kent Bazemore,20191119.0,17.6,"[19.0, 20.5225, 2.5, 6.75, 0.5, 2.0, 0.75, 1.2...","[25.7, 24.535, 3.5, 8.25, 1.25, 3.0, 1.0, 1.75...","[16.7, 19.8375, 2.5, 6.5, 1.0, 2.25, 1.0, 1.75...","[16.9, 17.8275, 2.25, 7.0, 1.0, 3.25, 0.5, 1.0...","[7.6, 15.1075, 2.75, 6.5, 1.5, 4.0, 0.25, 0.5,...","[32.6, 11.09, 1.75, 4.5, 1.0, 3.25, 0.0, 0.0, ...","[25.5, 18.2975, 3.0, 7.75, 1.75, 4.75, 0.0, 0....","[18.3, 20.8525, 3.0, 7.25, 1.75, 4.25, 1.25, 1...","[9.4, 18.635, 2.5, 6.5, 1.5, 3.5, 1.25, 1.5, 4...","[29.3, 21.7825, 3.0, 7.5, 1.5, 4.0, 1.5, 1.75,..."
1,Kent Bazemore,20191121.0,20.0,"[25.7, 24.535, 3.5, 8.25, 1.25, 3.0, 1.0, 1.75...","[16.7, 19.8375, 2.5, 6.5, 1.0, 2.25, 1.0, 1.75...","[16.9, 17.8275, 2.25, 7.0, 1.0, 3.25, 0.5, 1.0...","[7.6, 15.1075, 2.75, 6.5, 1.5, 4.0, 0.25, 0.5,...","[32.6, 11.09, 1.75, 4.5, 1.0, 3.25, 0.0, 0.0, ...","[25.5, 18.2975, 3.0, 7.75, 1.75, 4.75, 0.0, 0....","[18.3, 20.8525, 3.0, 7.25, 1.75, 4.25, 1.25, 1...","[9.4, 18.635, 2.5, 6.5, 1.5, 3.5, 1.25, 1.5, 4...","[29.3, 21.7825, 3.0, 7.5, 1.5, 4.0, 1.5, 1.75,...","[17.6, 18.765, 2.75, 7.75, 1.25, 3.75, 1.75, 2..."
2,Kent Bazemore,20191123.0,2.0,"[16.7, 19.8375, 2.5, 6.5, 1.0, 2.25, 1.0, 1.75...","[16.9, 17.8275, 2.25, 7.0, 1.0, 3.25, 0.5, 1.0...","[7.6, 15.1075, 2.75, 6.5, 1.5, 4.0, 0.25, 0.5,...","[32.6, 11.09, 1.75, 4.5, 1.0, 3.25, 0.0, 0.0, ...","[25.5, 18.2975, 3.0, 7.75, 1.75, 4.75, 0.0, 0....","[18.3, 20.8525, 3.0, 7.25, 1.75, 4.25, 1.25, 1...","[9.4, 18.635, 2.5, 6.5, 1.5, 3.5, 1.25, 1.5, 4...","[29.3, 21.7825, 3.0, 7.5, 1.5, 4.0, 1.5, 1.75,...","[17.6, 18.765, 2.75, 7.75, 1.25, 3.75, 1.75, 2...","[20.0, 17.6575, 2.75, 7.75, 1.75, 4.0, 1.0, 1...."
3,Kent Bazemore,20191125.0,6.6,"[16.9, 17.8275, 2.25, 7.0, 1.0, 3.25, 0.5, 1.0...","[7.6, 15.1075, 2.75, 6.5, 1.5, 4.0, 0.25, 0.5,...","[32.6, 11.09, 1.75, 4.5, 1.0, 3.25, 0.0, 0.0, ...","[25.5, 18.2975, 3.0, 7.75, 1.75, 4.75, 0.0, 0....","[18.3, 20.8525, 3.0, 7.25, 1.75, 4.25, 1.25, 1...","[9.4, 18.635, 2.5, 6.5, 1.5, 3.5, 1.25, 1.5, 4...","[29.3, 21.7825, 3.0, 7.5, 1.5, 4.0, 1.5, 1.75,...","[17.6, 18.765, 2.75, 7.75, 1.25, 3.75, 1.75, 2...","[20.0, 17.6575, 2.75, 7.75, 1.75, 4.0, 1.0, 1....","[2.0, 21.1375, 2.75, 8.75, 1.5, 4.0, 1.0, 1.25..."
4,Kent Bazemore,20191127.0,30.0,"[7.6, 15.1075, 2.75, 6.5, 1.5, 4.0, 0.25, 0.5,...","[32.6, 11.09, 1.75, 4.5, 1.0, 3.25, 0.0, 0.0, ...","[25.5, 18.2975, 3.0, 7.75, 1.75, 4.75, 0.0, 0....","[18.3, 20.8525, 3.0, 7.25, 1.75, 4.25, 1.25, 1...","[9.4, 18.635, 2.5, 6.5, 1.5, 3.5, 1.25, 1.5, 4...","[29.3, 21.7825, 3.0, 7.5, 1.5, 4.0, 1.5, 1.75,...","[17.6, 18.765, 2.75, 7.75, 1.25, 3.75, 1.75, 2...","[20.0, 17.6575, 2.75, 7.75, 1.75, 4.0, 1.0, 1....","[2.0, 21.1375, 2.75, 8.75, 1.5, 4.0, 1.0, 1.25...","[6.6, 19.935, 2.25, 8.5, 1.25, 3.75, 1.5, 1.75..."
